## Final month
There is one month to go in this competition. So many people have shared public data sets representing hours of hard work and now covering working solutions to all 400 problems. It would not have been possible for me to solve all 400, and I have also learned techniques from some of the solutions, notably [this notebook](https://www.kaggle.com/code/garrymoss/golfed-solutions-and-explanations-to-hard-problems) with code explanations from Garry Moss.


I am not a good golfer, but I do know that each hole is won or lost in the detail at the end. Many of the public solutions still need to be golfed down to something approaching the top task scores. So I thought I would share some techniques I have found helpful.

In this notebook I will explore the idea of rotating the grid and solving multiple times. I also hope to do future examples on recursion and regex pattern searches.

## Flip and zip
In many of the tasks, I have found it easier to solve from only one of the 4 edges, and then rotate. I will use `task 234` as an example of this technique.

In [ ]:
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *

In [ ]:
task_num = 234
show_examples(load_examples(task_num)['train'])
show_examples(load_examples(task_num)['arc-gen'][9:12])

## Filter out rows with one colour block
Using the yellow and purple example above, we could make a simple filter to remove rows that have exactly one colour block. Since we don't know what colour we are dealing with, it is better to count `w-1` black blocks.

In [ ]:
example=load_examples(task_num)["arc-gen"][9];g=example["input"]
w=len(g[0])
[r for r in g if w-1!=r.count(0)]

## Add blanks for the removed rows
Now we need to add back the blank rows. Then we can put the filter into a function and test it.

In [ ]:
def p(g):w=len(g[0]);u=[r for r in g if w-1!=r.count(0)];return[[0]*w]*(len(g)-len(u))+u

In [ ]:
example=load_examples(task_num)["arc-gen"][9];
solution={"input":example["input"],"output":p(example["input"])}
show_examples([example,solution])

## Partial solution
That simple `88 char` function works on about 25% of the examples.  
There are 2 problems:
1. Examples where the block must move up not down. The colour of the "rope" determines which block should move.
2. Examples that need to be filtered sideways across each row.
   

## Moving the block up
If the "rope" colour matches the bottom block. Then we have to move the bottom block up, rather than moving the top block down.  
The yellow and grey example `train[0]` is a good example of this

In [ ]:
example=load_examples(task_num)["train"][0];
solution={"input":example["input"],"output":p(example["input"])}
show_examples([example,solution])

## Modify the filter
First we need to know which colour block is on top. The code below picks out the top yellow block `colour code 4`. We then use this top colour to modify our filter so that rows are only removed if they match this top colour. Then we can safely add blank rows at the top to replace the remove rows.

In [ ]:
max([*filter(any,g)][0])

## Flip the grid upside down
Now that we have filtered properly, we need to flip the grid with `[::-1]` and repeat the filter operation. This then gives an algorithm that works for both examples as shown in the yellow blocks below.

In [ ]:
def p(g):
    for _ in'12':
        u=[r for r in g if r.count(max([*filter(any,g)][0]))!=1]
        g=((len(g)-len(u))*g[0:1]+u)[::-1]
    return g

In [ ]:
example=load_examples(task_num)["train"][0];
solution={"input":example["input"],"output":p(example["input"])}
show_examples([example,solution])

## The side-step
We now come to examples with sideways movement as illustrated by `train[1]` below

In [ ]:
example=load_examples(task_num)["train"][1];
solution={"input":example["input"],"output":p(example["input"])}
show_examples([example,solution])

## Transpose with zip
All these tasks would be greatly simplified if we were allowed to `import numpy` in this competition. Transposing a matrix is a good case in point. The best alternative I know of is to use zip. Using a list comprehension uses too many valuable characters, as shown in the comparison below.  
The results of both methods of transposing are the same. However, **as a word of caution**, zip returns tuples not lists. This needs to be changed back to lists for competition submission if I remember correctly.

In [ ]:
h=len(g);w=len(g[0]);R=range
t=[[g[r][c]for r in R(h)]for c in R(w)]
T=[*zip(*g)]
print(len(g),len(g[0]))
print(len(t),len(t[0]))
print(len(T),len(T[0]))

## Final algorithm
If we flip and zip (transpose) our grid and repeat the loop 4 times, then it passes on all the algorithms.

In [ ]:
def p(g):
    for _ in[0]*4:
        u=[r for r in g if r.count(max([*filter(any,g)][0]))!=1]
        g=[*zip(*((len(g)-len(u))*g[0:1]+u))][::-1]
    return g

In [ ]:
example=load_examples(task_num)["train"][1];
solution={"input":example["input"],"output":p(example["input"])}
show_examples([example,solution])

## Removing tuples for submission
**Version edit:** As pointed out by [jacekwl](https://www.kaggle.com/jacekwl) in the comments, the competition rules no longer require submission of lists only. Tuples are now also accepted. So the edit in the box below is not neccesary. The `Final algorithm` presented above is shorter and sufficient.  

Each row is changed back to a list with `[*r]`, which is no longer necessary for submission, but left in for reference and to keep the notebook versions consistant.

In [ ]:
%%writefile task.py
def p(g):
    for _ in[0]*4:
        u=[r for r in g if r.count(max([*filter(any,g)][0]))!=1]
        g=[*zip(*((len(g)-len(u))*g[0:1]+u))][::-1]
    return[[*r]for r in g]

## Version Edit: Folding back into a `lambda`
[Ches Charlemagne](https://www.kaggle.com/henrychibueze) did some gymnastics with the code structure to fold it all back into a single `lambda` function and reduce the byte count down to a competitive **131**. His code below.

In [ ]:
%%writefile task.py
p=lambda g:[(g:=[*zip(*((len(g)-len(u:=[r for r in g if r.count(max(next(filter(any,g))))!=1]))*g[:1]+u))][::-1])for _ in[0]*4][3]

In [ ]:
verify_program(task_num, load_examples(task_num))